In [1]:
#!pip install celery

In [2]:
#!sudo rabbitmqctl add_user barnabas barnabaspwd
#!sudo rabbitmqctl set_permissions -p / barnabas "." "." ".*"

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
barnabas	[]
guest	[administrator]
myguest	[]
myguest2	[]


on worker machine 1 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=3

on worker machine 2 start: <br>
celery -A test_workers worker --loglevel=info --concurrency=2


In [4]:
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import json

import celery
from test_workers import message_to_workers1

import math
import socket


In [5]:
celery.current_app.control.inspect().ping()

{'celery@ip-172-31-0-150': {'ok': 'pong'},
 'celery@ip-172-31-3-73': {'ok': 'pong'}}

In [6]:
mykeys=list(celery.current_app.control.inspect().stats().keys())
print(mykeys)

['celery@ip-172-31-0-150', 'celery@ip-172-31-3-73']


In [7]:
socket.gethostbyname(socket.gethostname())

'172.31.0.150'

In [8]:
mydict=celery.current_app.control.inspect().stats()[mykeys[0]]
mydict['pool']

{'max-concurrency': 3,
 'processes': [17792, 17793, 17794],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 3, 'active': 0}}}

In [9]:
mydict=celery.current_app.control.inspect().stats()[mykeys[1]]
mydict['pool']

{'max-concurrency': 4,
 'processes': [12392, 12393, 12394, 12396],
 'max-tasks-per-child': 'N/A',
 'put-guarded-by-semaphore': False,
 'timeouts': [0, 0],
 'writes': {'total': 0,
  'avg': '0.00%',
  'all': '',
  'raw': '',
  'strategy': 'fair',
  'inqueues': {'total': 4, 'active': 0}}}

In [10]:
celery.current_app.pool

In [11]:
for iter in range(0,30):
    myresult=message_to_workers1.delay(str(1000+iter)) 
    results=myresult.get()
    print('***')
    print(results)
    print('***')

***
['172.31.0.150_ForkPoolWorker-2_1_1000_69_55071']
***
***
['172.31.3.73_ForkPoolWorker-2_1_1001_640_55082']
***
***
['172.31.0.150_ForkPoolWorker-3_2_1002_69_55090']
***
***
['172.31.3.73_ForkPoolWorker-1_0_1003_640_55098']
***
***
['172.31.0.150_ForkPoolWorker-1_0_1004_69_55101']
***
***
['172.31.3.73_ForkPoolWorker-2_1_1005_12_55109', '172.31.3.73_ForkPoolWorker-2_1_1001_640_55082']
***
***
['172.31.0.150_ForkPoolWorker-2_1_1006_954_55112', '172.31.0.150_ForkPoolWorker-2_1_1000_69_55071']
***
***
['172.31.3.73_ForkPoolWorker-1_0_1007_12_55116', '172.31.3.73_ForkPoolWorker-1_0_1003_640_55098']
***
***
['172.31.0.150_ForkPoolWorker-3_2_1008_954_55122', '172.31.0.150_ForkPoolWorker-3_2_1002_69_55090']
***
***
['172.31.3.73_ForkPoolWorker-2_1_1009_200_55124', '172.31.3.73_ForkPoolWorker-2_1_1005_12_55109', '172.31.3.73_ForkPoolWorker-2_1_1001_640_55082']
***
***
['172.31.0.150_ForkPoolWorker-1_0_1010_954_55128', '172.31.0.150_ForkPoolWorker-1_0_1004_69_55101']
***
***
['172.31.3.73_F